<a href="https://colab.research.google.com/github/Ozaira-Hameed/Streamlit-LLM-Interface/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
"""
RAG WITH UNSLOTH - FIXED FOR T4 GPU
Uses smaller model that fits in 15GB VRAM
"""

# ============================================================================
# INSTALLATION
# ============================================================================
print("Installing packages...")
!pip install -q unsloth
!pip install -q sentence-transformers
!pip install -q chromadb
print("✅ Installation complete!\n")

# ============================================================================
# IMPORTS
# ============================================================================
import torch
import gc
from unsloth import FastLanguageModel
from sentence_transformers import SentenceTransformer
import chromadb
from typing import List
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported!\n")

# ============================================================================
# YOUR DOCUMENTS - EDIT THIS! 👇
# ============================================================================
DOMAIN_DOCUMENTS = [
    """
    Artificial Intelligence in Healthcare:
    AI is revolutionizing healthcare through diagnostic tools, personalized medicine,
    and patient care automation. Machine learning algorithms analyze patient data to
    predict disease outcomes with 95% accuracy. Computer vision detects tumors in
    medical imaging faster than radiologists. NLP extracts insights from clinical notes.
    AI chatbots provide 24/7 patient support and preliminary diagnoses.
    """,

    """
    Retrieval-Augmented Generation (RAG):
    RAG combines information retrieval with text generation for accurate responses.
    The system retrieves relevant documents using semantic search, then uses them as
    context for the language model. This reduces hallucinations and provides verifiable
    sources. RAG is effective for question-answering, summarization, and knowledge-
    intensive tasks where accuracy is critical.
    """,

    """
    Model Quantization:
    Quantization reduces model size by using lower precision numbers. Dynamic 4-bit
    quantization preserves precision for critical parameters while compressing others.
    This reduces model size by 75% while maintaining 95-98% performance. Benefits
    include faster inference, lower memory usage, and ability to run large models
    on consumer hardware.
    """,

    """
    Vector Databases:
    Vector databases store high-dimensional embeddings for fast similarity search.
    Embeddings capture semantic meaning, enabling retrieval of conceptually similar
    content without exact keyword matches. Modern models produce 384-768 dimensional
    vectors. ANN algorithms enable sub-millisecond search across millions of vectors.
    Popular options include ChromaDB, Pinecone, and FAISS.
    """,

    """
    Transformer Architecture:
    Transformers revolutionized NLP through self-attention mechanisms. Unlike RNNs,
    they process entire sequences in parallel for faster training. Attention learns
    relationships between all words, capturing long-range dependencies. Multi-head
    attention focuses on different aspects simultaneously. This powers GPT, BERT,
    and LLaMA models.
    """,
]

print(f"📚 Loaded {len(DOMAIN_DOCUMENTS)} documents\n")

# ============================================================================
# DOCUMENT PROCESSING
# ============================================================================
def chunk_documents(documents: List[str], chunk_size: int = 512) -> List[str]:
    """Split documents into chunks"""
    all_chunks = []

    for doc in documents:
        doc = doc.strip()
        start = 0

        while start < len(doc):
            end = start + chunk_size

            if end < len(doc):
                space = doc.rfind(' ', start, end)
                if space != -1:
                    end = space

            chunk = doc[start:end].strip()
            if chunk:
                all_chunks.append(chunk)

            start = end - 50  # overlap

    return all_chunks

chunks = chunk_documents(DOMAIN_DOCUMENTS)
print(f"✅ Created {len(chunks)} chunks\n")

# ============================================================================
# LOAD SMALLER MODEL (FITS IN T4 GPU!)
# ============================================================================
print("🤖 Loading smaller Unsloth model for T4 GPU...\n")

# TRY OPTION 1: Gemma 2B (smallest, most likely to work)
try:
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name="unsloth/gemma-2-2b-it-bnb-4bit",  # Smaller model!
        max_seq_length=1024,  # Reduced sequence length
        dtype=None,
        load_in_4bit=True,
    )
    FastLanguageModel.for_inference(model)
    print("✅ Loaded Gemma-2B model successfully!\n")
    model_loaded = True

except Exception as e:
    print(f"❌ Gemma failed: {e}\n")
    model_loaded = False

# TRY OPTION 2: Phi-3 Mini (backup option)
if not model_loaded:
    try:
        print("Trying Phi-3 Mini model...\n")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name="unsloth/Phi-3.5-mini-instruct-bnb-4bit",
            max_seq_length=1024,
            dtype=None,
            load_in_4bit=True,
        )
        FastLanguageModel.for_inference(model)
        print("✅ Loaded Phi-3 Mini model successfully!\n")
        model_loaded = True
    except Exception as e:
        print(f"❌ Phi-3 failed: {e}\n")
        model_loaded = False

# TRY OPTION 3: TinyLlama (smallest possible)
if not model_loaded:
    try:
        print("Trying TinyLlama model...\n")
        model, tokenizer = FastLanguageModel.from_pretrained(
            model_name="unsloth/tinyllama-bnb-4bit",
            max_seq_length=1024,
            dtype=None,
            load_in_4bit=True,
        )
        FastLanguageModel.for_inference(model)
        print("✅ Loaded TinyLlama model successfully!\n")
        model_loaded = True
    except Exception as e:
        print(f"❌ TinyLlama failed: {e}\n")
        model_loaded = False

if not model_loaded:
    raise RuntimeError("Could not load any model. GPU might not have enough memory.")

# Show memory usage
if torch.cuda.is_available():
    vram = torch.cuda.memory_allocated() / 1e9
    total = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ Model loaded!")
    print(f"   VRAM used: {vram:.2f} GB / {total:.2f} GB\n")

# ============================================================================
# CREATE VECTOR DATABASE
# ============================================================================
print("🔍 Creating vector search system...\n")

embedder = SentenceTransformer('all-MiniLM-L6-v2')
db = chromadb.Client()
collection = db.get_or_create_collection(name="docs")

# Index chunks
for i, chunk in enumerate(chunks):
    embedding = embedder.encode(chunk).tolist()
    collection.add(
        embeddings=[embedding],
        documents=[chunk],
        ids=[f"chunk_{i}"]
    )

print(f"✅ Indexed {len(chunks)} chunks!\n")

# ============================================================================
# RAG FUNCTIONS
# ============================================================================

def retrieve_context(query: str, top_k: int = 3) -> List[str]:
    """Retrieve relevant chunks"""
    query_embedding = embedder.encode(query).tolist()
    results = collection.query(
        query_embeddings=[query_embedding],
        n_results=top_k
    )
    return results['documents'][0]

def generate_answer(query: str, context: List[str]) -> str:
    """Generate answer using context"""

    context_text = "\n\n".join([f"[{i+1}] {c}" for i, c in enumerate(context)])

    prompt = f"""Based on the context, answer the question.

Context:
{context_text}

Question: {query}

Answer:"""

    # Tokenize with shorter max length
    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=800  # Reduced to fit in memory
    ).to("cuda")

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=150,  # Reduced token generation
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Extract answer
    if "Answer:" in response:
        answer = response.split("Answer:")[-1].strip()
    else:
        # Fallback: take everything after the prompt
        answer = response[len(prompt):].strip()

    # Memory cleanup
    del inputs, outputs
    gc.collect()
    torch.cuda.empty_cache()

    return answer

def ask_custom_question(question: str, show_sources: bool = True):
    """Ask a question to the RAG system"""

    print("\n" + "="*70)
    print("RAG QUERY")
    print("="*70)
    print(f"\n🤔 Question: {question}\n")

    # Retrieve
    context = retrieve_context(question)

    if show_sources:
        print("📚 Retrieved Sources:")
        for i, source in enumerate(context, 1):
            preview = source[:80] + "..." if len(source) > 80 else source
            print(f"   [{i}] {preview}")
        print()

    # Generate
    print("💭 Generating answer...\n")
    answer = generate_answer(question, context)

    print("💡 Answer:")
    print(f"   {answer}")
    print("\n" + "="*70 + "\n")

    return answer

def ask_multiple_questions(questions: List[str]):
    """Ask multiple questions"""

    print(f"\n{'='*70}")
    print(f"PROCESSING {len(questions)} QUESTIONS")
    print('='*70)

    results = []

    for i, question in enumerate(questions, 1):
        print(f"\n[{i}/{len(questions)}] {question}")
        print("-"*70)

        context = retrieve_context(question)
        answer = generate_answer(question, context)

        print(f"Answer: {answer}")

        results.append({
            'question': question,
            'answer': answer
        })

    print(f"\n{'='*70}")
    print(f"✅ COMPLETED {len(results)} QUESTIONS")
    print('='*70 + "\n")

    return results

def show_memory():
    """Show GPU memory usage"""
    if torch.cuda.is_available():
        allocated = torch.cuda.memory_allocated() / 1e9
        reserved = torch.cuda.memory_reserved() / 1e9
        total = torch.cuda.get_device_properties(0).total_memory / 1e9
        print(f"\n📊 GPU Memory:")
        print(f"   Used: {allocated:.2f} GB / {total:.2f} GB")
        print(f"   Reserved: {reserved:.2f} GB\n")

# ============================================================================
# RUN DEMO
# ============================================================================

print("\n" + "="*70)
print("RUNNING DEMO QUERIES")
print("="*70)

demo_queries = [
    "How is AI used in healthcare?",
    "What is RAG?",
    "Explain quantization",
]

for query in demo_queries:
    ask_custom_question(query, show_sources=True)

show_memory()

# ============================================================================
# READY!
# ============================================================================

print("="*70)
print("✅ RAG SYSTEM READY!")
print("="*70)
print("""
NOW ASK YOUR QUESTIONS!

USAGE:
    ask_custom_question("Your question?")

EXAMPLES:
    ask_custom_question("What are vector databases?")
    ask_custom_question("How do transformers work?")

MULTIPLE QUESTIONS:
    questions = ["Q1?", "Q2?", "Q3?"]
    ask_multiple_questions(questions)

CHECK MEMORY:
    show_memory()
""")
print("="*70)

# ============================================================================
# YOUR QUESTIONS HERE! 👇
# ============================================================================

# Uncomment to test:
# ask_custom_question("What is artificial intelligence?")

Installing packages...
✅ Installation complete!

✅ Libraries imported!

📚 Loaded 5 documents

✅ Created 5 chunks

🤖 Loading smaller Unsloth model for T4 GPU...

==((====))==  Unsloth 2026.1.4: Fast Gemma2 patching. Transformers: 4.57.6.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.10.0+cu128. CUDA: 7.5. CUDA Toolkit: 12.8. Triton: 3.6.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.34. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.22G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

✅ Loaded Gemma-2B model successfully!

✅ Model loaded!
   VRAM used: 2.29 GB / 15.83 GB

🔍 Creating vector search system...

✅ Indexed 5 chunks!


RUNNING DEMO QUERIES

RAG QUERY

🤔 Question: How is AI used in healthcare?

📚 Retrieved Sources:
   [1] Artificial Intelligence in Healthcare:
    AI is revolutionizing healthcare thro...
   [2] Transformer Architecture:
    Transformers revolutionized NLP through self-atten...
   [3] Retrieval-Augmented Generation (RAG):
    RAG combines information retrieval wit...

💭 Generating answer...

💡 Answer:
   AI is used in healthcare in various ways such as: 
 *  **Diagnosis:** AI-powered diagnostic tools predict disease outcomes with high accuracy.
 *  **Personalized Medicine:** AI analyzes patient data to create personalized treatments. 
 *  **Patient Care Automation:** AI chatbots provide 24/7 support and preliminary diagnoses. 
 *  **Medical Imaging:** Computer Vision detects tumors in medical images faster than radiologists. 
 *  **Clinical 

In [8]:
# Test 1
ask_custom_question("What is artificial intelligence?")

# Test 2
ask_custom_question("How does RAG work?")

# Test 3
my_questions = [
    "What is quantization?",
    "What are transformers?"
]
ask_multiple_questions(my_questions)


RAG QUERY

🤔 Question: What is artificial intelligence?

📚 Retrieved Sources:
   [1] Artificial Intelligence in Healthcare:
    AI is revolutionizing healthcare thro...
   [2] Transformer Architecture:
    Transformers revolutionized NLP through self-atten...
   [3] Retrieval-Augmented Generation (RAG):
    RAG combines information retrieval wit...

💭 Generating answer...

💡 Answer:
   Artificial intelligence is revolutionizing healthcare through diagnostic tools, personalized medicine, and patient care automation. Machine learning algorithms analyze patient data to predict disease outcomes with 95% accuracy. Computer vision detects tumors in medical imaging faster than radiologists. NLP extracts insights from clinical notes. AI chatbots provide 24/7 patient support and preliminary diagnoses.



RAG QUERY

🤔 Question: How does RAG work?

📚 Retrieved Sources:
   [1] Retrieval-Augmented Generation (RAG):
    RAG combines information retrieval wit...
   [2] Transformer Architecture:
    

[{'question': 'What is quantization?',
  'answer': 'Quantization is a technique that reduces model size by using lower precision numbers, especially to compress other parameters while preserving precision for critical parameters. This results in a 75% reduction in model size while maintaining 95-98% performance.'},
 {'question': 'What are transformers?',
  'answer': '** \n\nThe answer is directly taken from the first paragraph of the provided text on "Transformer Architecture."  This answer succinctly and accurately describes what transformers are.'}]